In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import tensorflow as tf

from lfm_flow.mogaussianprocesses import MultioutputGaussianProcess
from lfm_flow.mogaussianprocesses import MultioutputGaussianProcessRegressionModel
from lfm_flow.kernels import LFM1_RBF
from lfm_flow.latentforcemodels import LatentForceModel

np.random.seed(123)  # reproducability!


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
def reset_session():
  """Creates a new global, interactive session in Graph-mode."""
  global sess
  try:
    tf.reset_default_graph()
    sess.close()
  except:
    pass
  sess = tf.InteractiveSession()

reset_session()

In [15]:
D = np.array([1., 0.3, -0.2])
S = np.array([[.33],
              [-0.4]])
length_scales = np.array([1., 0.95])

kern = LFM1_RBF(D, S, length_scales)


In [16]:
t1 = np.linspace(0., 3., 5)[:, None]

index_points = (t1, )
lf_index_points = (t1, None)

lfm = LatentForceModel(kern, index_points, 
                       lf_index_points=lf_index_points)
print(lfm.lf_index_points_shape)

[5, None]


In [17]:
from tensorflow_probability import distributions as tfd
from tensorflow_probability import positive_semidefinite_kernels as tfk

In [39]:

class FlattenedLFMKernel(tfk.PositiveSemidefiniteKernel):
    def __init__(self, 
                 lfm_kernel, 
                 index_points_shape,
                 lf_index_points_shape=(0, )):
        """
        Allows for reshaping of the 
        """
        self._lfm_kernel = lfm_kernel
        self._index_points_shape = index_points_shape
        self._lf_index_points_shape = lf_index_points_shape
        
    @property
    def output_kernel(self):
        return self._lfm_kernel
        
    @property
    def index_points_shape(self):
        return self._index_points_shape
    
    @property
    def lf_index_points_shape(self):
        return self._lf_index_points_shape
    
    def _apply(self, x1, x2, param_expansion_ndims=0):
        
        # x1 will be shape [b1,...,bB, e1, 1]
        # x2 will be shape [b1,...,bB, e2, 1]
        
        n_output = sum(self.index_points_shape)
        n_lfs = sum(self.lf_index_points_shape)
        
        x1_outputs, x1_lfs = tf.split(x1, (n_output, n_lfs), axis=-2,)
       

x1_index_points = np.linspace(0., 3., 5)[:, None]
x2_index_points = None
x3_index_points = np.linspace(0., 2., 3)[:, None]

lf1_index_points = np.linspace(0., 3.5, 6)[:, None]


foo = FlattenedLFMKernel(kern, [5, 0, 3], [6, ])



flat_x1 = tf.concat([x1_index_points,
                     x3_index_points,
                     lf1_index_points],
                    axis=-2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    foo._apply(flat_x1, None)

        

(6, 1)


### GradientGaussianProcess

```MultioutputGaussianProcess```, the number of outputs is determined by the dimension of the feature space

$$
u_{p}(t) := \frac{\partial}{\partial x_p} y(\mathbf{x})
$$

We also take $u_{0}(t)$ to be the original function, so that
```
>>> length_scale = np.array([1.])
>>> kern = ExponentiatedQuadraticGradient(length_scale)
>>> kern.output_ndim = 2
```

So that

```
index_points = ...
gradient_index_points = ...

K = kern.matrix((index_points, gradient_index_points), 
                (index_points, gradient_index_points))
     _           _
    |             |
    | Kff   Kff'  |
K = |             |
    | Kf'f  Kf'f' |
    |_           _|
```


In [ ]:
class GradientKernel:
    """
    
    """
    pass

class GradientGaussianProcess:
    def __init__(self,
                 kernel,
                 gradient_kernel,
                 index_points,
                 gradient_index_points):
        pass
    
class GradientGaussianProcessRegressionModel:
    def __init__(self, 
                 kernel, 
                 gradient_kernel,
                 index_points,
                 gradient_index_points,
                 observation_index_points=None,
                 observations=None,
                 gradient_observation_index_points=None,
                 gradient_observations=None
                 )
        pass
    